In [1]:
import os

In [3]:
%pwd

'/Users/snehalmachan/Coding/textsummarizer/research'

In [4]:
os.chdir("../")
%pwd

'/Users/snehalmachan/Coding/textsummarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

#below is an entity that defines the return type of a function
@dataclass(frozen= True) #(not a python class but a dataclass)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [58]:
import textsummarizer.constants.constants as constants
from textsummarizer.utils.common import read_yaml, create_directories


In [73]:

class ConfigurationManager:
    def __init__(
            self,
            params_file_path: Path= constants.PARAMS_FILE_PATH,
            config_file_path: Path= constants.CONFIG_FILE_PATH,):
        
        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)

        create_directories([self.config.artifacts_root,])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir =  config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config


In [62]:
import os
import urllib.request as request 
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size



In [64]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                    url= self.config.source_URL,
                    filename=  self.config.local_data_file 
            ) 
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [74]:
###Creating the Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-26 19:46:52,519: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-26 19:46:52,524: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-26 19:46:52,543: INFO: common: created directory at :artifacts]


[2023-11-26 19:46:52,556: INFO: common: created directory at :artifacts/data_ingestion]
[2023-11-26 19:47:27,488: INFO: 2662514189: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 24441500
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "165933e50795792296f76b85aaf91d3eb23e87d0eae076c6e4098708d6f973f4"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 73B0:B8CFB:2186FB:3B75AD:65635355
Accept-Ranges: bytes
Date: Sun, 26 Nov 2023 14:16:55 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100035-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1701008214.771950,VS0,VE1542
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: e9fbf1b060a4477549bd7c3b34c5493e97cf861b
Expires: